In [1]:
import numpy as np
import random
import itertools 
from collections import Counter

In [2]:
hierarchy_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']
suits = ['♠', '♦', '♥', '♣']
cards = [(x,y) for x in hierarchy_values for y in suits]
random.shuffle(cards)

def deal(cards):
    dealt_cards = []
    for i in range(7):
        dealt_cards.append(cards.pop(0))
    return dealt_cards



In [3]:
def translate(card):
    if card == 'J':
        return 11
    if card == 'Q':
        return 12
    if card == 'K':
        return 13
    if card == 'A':
        return 14
    else:
        return card
def untranslate(value):
    if value == 11:
        return 'J'
    if value == 12:
        return 'Q'
    if value == 13:
        return 'K'
    if value == 14:
        return 'A'
    else:
        return value

In [4]:
def evaluate_bottom(cards):


#     Royal Flush (A-K-Q-J-10 of any one suit) -- 10
#     Straight Flush (5 consecutive cards of any one suit) -- 9
#     Four-of-a-kind -- 8
#     Full House -- 7
#     Flush -- 6
#     Straight -- 5
#     Three-of-a-kind -- 4
#     Two Pair -- 3
#     One Pair -- 2
#     High Card -- 1
    suits = []
    values = []
    temp = cards
    possible_values = ['A',2,3,4,5,6,7,8,9,10, 11, 12, 13]
    
        
    for x, y in temp:
        values.append(x)
        suits.append(y)
        
        
    current_max = 0
        

    
    #royal flush
    royal_flush_set = {10, 'J', 'Q', 'K', 'A'}
    if set(values).intersection(royal_flush_set) == 5:

        suits_rf = Counter(suits).most_common()
        
        if [count for suit, count in suits_rf].count(5) == 1:
            return 10
        
    

    #four of a kind
 
    value_counts = Counter(values).most_common()
    if [count for card, count in value_counts].count(4) == 1:
        return 8
    
    #full house
    if [count for card, count in value_counts].count(3) == 1 and [count for card, count in value_counts].count(2) == 1:
        return 7
    
    #flush
    suit_counts = Counter(suits).most_common()
    suits = []

    if [count for card, count in suit_counts].count(5) == 1:
        return 6
       
        
    #straight
    straight_temp = values
    straight_temp = [translate(x) for x in straight_temp]


    if 'A' in straight_temp:
            need_high = [10, 11, 12, 13]
            need_low = [2, 3, 4, 5]
            result_high =  all(elem in straight_temp for elem in need_high)
            result_low =  all(elem in straight_temp for elem in need_low)

            if result_high or result_low:
                return 5 #A2345 or 10JQKA
                pass
    else:      
        straight_temp = sorted(straight_temp)
        for i in range(len(straight_temp)):
            if straight_temp[i] < 10:
                need_list = list(range(straight_temp[i]+1,straight_temp[i]+5))

                if all(item in straight_temp for item in need_list):
                    return 5
                    pass

    
    
    
    
   
    #three of a kind 

    value_counts = Counter(values).most_common()
    if [count for card, count in value_counts].count(3) == 1:
        return 4       
    
    #two pair 
    value_counts = Counter(values).most_common()

    if [count for card, count in value_counts].count(2) == 2:
        return 3
    
    #one pair
    if [count for card, count in value_counts].count(2) == 1:
        return 2 
    
    #high card
    if [count for card, count in value_counts].count(2) == 0:
        return 1
    


In [21]:
def compare_two_cards(hand1, hand2):

    values1, values2 = [], []
    for x, y in hand1:
        values1.append(x)
    values1 = [translate(x) for x in values1]

    for x, y in hand2:
        values2.append(x)
    values2 = [translate(x) for x in values2]

    #set of two pairs    
    if values1[0] == values1[1] and values2[0] == values2[1]:
        if values1[0] >= values2[0]:
            return hand1
        else:
            return hand2
    #hand1 has pair and hand 2 doesn't
    if values1[0] == values1[1] and values2[0] != values2[1]:
        return hand1
    #hand2 has pair and hand1 doesn't
    if values1[0] != values1[1] and values2[0] == values2[1]:
        return hand2
    #no pairs
    if values1[0] != values1[1] and values2[0] != values2[1]:
        high1, high2 = max(values1), max(values2)
        if high1 > high2:
            return hand1
        elif high2 > high1:
            return hand2
        else:
            min1, min2 = min(values1), min(values2)
            if min1 >= min2:
                return hand1
            else:
                return hand2
        
            
    
        

In [41]:

def evaluate_seven(dealt_cards):
    #weird tricky exception for 3 pairs here.

    dealt_cards = sorted(dealt_cards, key=lambda x: hierarchy_values.index(x[0]), reverse = True)

    hand_of_five = []
    hand_of_two = []
    curr_max = 0

    temp = dealt_cards
    values = []
    suits = []
    
  
    for x, y in temp:
        values.append(x)
        suits.append(y)    

    
    value_counts = Counter(values).most_common() 
    
    ### THREE PAIR IS AN EXCEPTION TO RULES WE MUST LOOK AT FIRST 
    
    if [count for card, count in value_counts].count(2) == 3:
        pairs = []
        singles = []
        for x, y in value_counts:
            if y == 1:
                singles.append(x)
            if y == 2:
                pairs.append(x)

        singles = [translate(x) for x in singles]

        pairs = [translate(x) for x in pairs]

        hand_of_two = set(filter(lambda c: c[0] == max(pairs), dealt_cards))
        hand_of_five = tuple(set(dealt_cards).difference(set(hand_of_two)))
        return hand_of_five, tuple(hand_of_two)
        
    
    #LOOP STARTS HERE
    
    card_list = list(itertools.combinations(dealt_cards, 5))
    for i in range(21):

        if evaluate_bottom(card_list[i]) > curr_max:

            curr_max = evaluate_bottom(card_list[i])
            hand_of_five = card_list[i]
            hand_of_two = tuple(set(dealt_cards).difference(set(hand_of_five)))
            top_hand = hand_of_five, hand_of_two

        else:
            #print('tie')
            
            #high card
            if curr_max == 1:
  
                #normal pai gow
                old_hand_values = hand_of_five
                oh_values = []
                for x,y in old_hand_values:
                    oh_values.append(x)
                oh_values = [translate(x) for x in oh_values]


                new_hand_values = card_list[i]
                nh_values = []
                for x,y in new_hand_values:
                    nh_values.append(x)
                    nh_values = [translate(x) for x in nh_values]

                
                #this ensures we keep the high card in the 5 card hand
                if max(oh_values) > max(nh_values):
                    pass
                
                else:
                    new_hand_of_two = tuple(set(dealt_cards).difference(set(card_list[i])))

                    winner_of_twos = compare_two_cards(hand_of_two, new_hand_of_two)
                    if winner_of_twos == new_hand_of_two:
                        hand_of_two = new_hand_of_two
                        hand_of_five = tuple(set(dealt_cards).difference(set(hand_of_two)))
                        #print('new winner! 2')
                        #print(hand_of_five, hand_of_two)
                    
                    
            
            elif curr_max == 3:

                practice_hand = dealt_cards
                practice_values = []
                for x, y in practice_hand:
                    practice_values.append(x)
                practice_counts = Counter(practice_values).most_common()

                pairs = []
                singles = []
                for x, y in practice_counts:
                    if y == 1:
                        singles.append(x)
                    if y == 2:
                        pairs.append(x)

                singles = [translate(x) for x in singles]

                pairs = [translate(x) for x in pairs]


                if max(singles) == 14:

                    max_ = set(filter(lambda c: c[0] == untranslate(max(singles)) , practice_hand))
                    second_ = set(filter(lambda c: c[0] == int(np.median(singles)), practice_hand))
                    hand_of_two = tuple(max_.union(second_))
                else:
                    hand_of_two = tuple(set(filter(lambda c: c[0] == untranslate(min(pairs)), practice_hand)))

                hand_of_five = tuple(set(dealt_cards).difference(set(hand_of_two)))
                return tuple(hand_of_five), tuple(hand_of_two)
                break
                
 

            elif curr_max == 7:
                practice_hand = dealt_cards
                practice_values = []
                for x, y in practice_hand:
                    practice_values.append(x)
                practice_counts = Counter(practice_values).most_common()
                
                trips = []
                pairs = []
                singles = []
                
                for x, y in practice_counts:
                    if y == 1:
                        singles.append(x)
                    if y == 2:
                        pairs.append(x)
                    if y == 3:
                        trips.append(x)

                singles = [translate(x) for x in singles]

                pairs = [translate(x) for x in pairs]
                
                trips = [translate(x) for x in pairs]
                
                if len(pairs) == 2:
                    larger_pair = max(pairs)
                    hand_of_two = set(filter(lambda c: c[0] == untranslate(min(pairs)), practice_hand))
                    hand_of_five = tuple(set(dealt_cards).difference(set(hand_of_two)))
                    return tuple(hand_of_five), tuple(hand_of_two)
                else:
                    if max(singles) == 14:
                        hand_of_two = set(filter(lambda c: c[0] == untranslate(singles), practice_hand))
                        hand_of_five = tuple(set(dealt_cards).difference(set(hand_of_two)))
                    else:
                        hand_of_two = set(filter(lambda c: c[0] == untranslate(pairs[0]), practice_hand))
                        hand_of_five = tuple(set(dealt_cards).difference(set(hand_of_two)))
                    return tuple(hand_of_five), tuple(hand_of_two)
                
                    
                
                
            ## ELSE
            
            elif evaluate_bottom(card_list[i]) == curr_max:

                new_hand_of_two = tuple(set(dealt_cards).difference(set(card_list[i])))

                winner_of_twos = compare_two_cards(hand_of_two, new_hand_of_two)
                if winner_of_twos == new_hand_of_two:
                    hand_of_two = new_hand_of_two
                    hand_of_five = tuple(set(dealt_cards).difference(set(hand_of_two)))

                    
            
                    
               
    return tuple(hand_of_five), tuple(hand_of_two)
                
        
        
        


In [9]:
# evaluate_seven(((8, '♦'), (7, '♣'), (7, '♥'), (8, '♣'), (6, '♦'), (6, '♥'), ('A', '♦')))